In [8]:
usr = 'vokulovskiy'
pwd = 'OsPl27FLJC7c8N6t'
host = '212.24.35.176'
port = 10573

In [11]:
import psycopg2
import pandas as pd
import re

# pip install psycopg2-binary
# poetry add psycopg2-binary

min_price = 50000
usr = 'vokulovskiy'
pwd = 'OsPl27FLJC7c8N6t'
host = '212.24.35.176'
port = 10573

def search_keys(text, keys):
    '''
    Функция считает количество ключевых слов/фраз в тексте
    На входе - текст, список ключевых слов в виде списка фраз в формате regex
    На выходе - словарь с ключами и их количеством
    '''
    text =  text.lower().strip() # переводим в нижний регистр
    text = ' '.join(text.split()) # удаляем двойные пробелы
    cnt= {l[0]:len(l) for k in keys if len(l:=re.findall(k.lower(), text))>0}       
    return cnt

try:
    # пытаемся подключиться к базе данных
    conn = psycopg2.connect(dbname='attractor-raw', user=usr, password=pwd, host=host, port=port)
    print('Connected!')
except:
    # в случае сбоя подключения будет выведено сообщение  в STDOUT
    print('Can`t establish connection to database')

conn.autocommit = True  # устанавливаем актокоммит

df_db = pd.read_sql("SELECT sk,title,price,fit FROM gz44_ord WHERE fit=0", conn)
df_keys = pd.read_csv('key_words.csv',delimiter=';')
# Negative keys
keys_n = df_keys[(df_keys.np==0)].key.to_list()
# Positive keys
keys_p = df_keys[(df_keys.np==1)].key.to_list()

with conn.cursor() as curs:
    for idx,row in df_db.iterrows():
        if row.price < min_price:
            row.fit = -1
        else:
            try:
                text =  row.title.lower().strip() # переводим в нижний регистр
                count_n = len(search_keys(text, keys_n)) 
                count_p = len(search_keys(text, keys_p)) 
                if count_p==0 and count_n > 0:
                    row.fit = -1
                else:
                    row.fit = 1
            except:
                row.fit = -1
        print(row.fit, row.price, row.title)
        sql = f"UPDATE gz44_ord SET fit = {row.fit} WHERE sk = {row.sk};"
        curs.execute(sql)
print('='*100)
df = pd.read_sql("SELECT sk,title,price,fit FROM gz44_ord WHERE fit!=0", conn)
print(df.shape)
conn.close()

Connected!


/tmp/ipykernel_194572/3066469540.py:35: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df_db = pd.read_sql("SELECT sk,title,price,fit FROM gz44_ord WHERE fit=0", conn)


-1 202603.1 Поставка неисключительных прав на программное обеспечение для нужд Клиники ФГБОУ ВО БГМУ Минздрава России
1 95028.0 Сопровождение Электронного периодического справочника «Система ГАРАНТ», содержащего информацию о текущем состоянии законодательства Российской Федерации, путем предоставления текущих версий комплекта ЭПС «Система ГАРАНТ»
-1 123666.9 Оказание консультационных услуг по использованию программного обеспечения 1С: Бухгалтерия государственного учреждения 8.3.
1 21552000.0 Услуги по созданию единого телефонного центра МФЦ Ростовской области
-1 99000.0 Предоставление консультационных услуг по эксплуатации системы электронного документооборота Ленинградской области в 2024 году
-1 266631.03 Оказание услуг по сопровождению и обновлению Электронного периодического справочника «Система ГАРАНТ»
-1 127200.0 Услуги по  технической поддержке находящегося в эксплуатации Заказчика программного обеспечения  системы электронного документооборота «ёЛогика» (СЭД ёЛогика)
-1 108145.1

In [9]:
try:
    # пытаемся подключиться к базе данных
    conn = psycopg2.connect(dbname='attractor-raw', user=usr, password=pwd, host=host, port=port)
    print('Connected!')
except:
    # в случае сбоя подключения будет выведено сообщение  в STDOUT
    print('Can`t establish connection to database')

df = pd.read_sql("SELECT sk,title,price,fit FROM gz44_ord", conn)
print(df.shape)

Connected!


/tmp/ipykernel_194572/3738818149.py:9: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df = pd.read_sql("SELECT sk,title,price,fit FROM gz44_ord", conn)


(27628, 4)


In [10]:
conn.close()